In [ ]:
# !pip install matplotlib
import matplotlib.pyplot as plt
import os
import numpy as np
!pip install tensorflow 
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from tensorflow.keras import layers
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
# from tensorflow.keras.utils import to_categorical
%matplotlib inline

In [ ]:
# Get Fish categories available in the root dir
root_dir = "Fish images"
train_image_dir = os.path.join(root_dir, "train")
val_image_dir = os.path.join(root_dir, "val")

labels = []
for label in os.listdir(train_image_dir):
    labels.append(label)
    
display(labels)

### Loading the image files

In [ ]:
# Loading the data
!pip install opencv-python
import cv2
img_size = 150
image_data = []

def get_data(image_dir):
    for label in labels:
        image_path = os.path.join(image_dir, label)
        class_num = labels.index(label)  # class num gives each fish category a label
        for img in os.listdir(image_path):
            img_arr = cv2.imread(os.path.join(image_path, img))
    #             resize the images
            resized_images = cv2.resize(img_arr, (img_size, img_size))
            image_data.append([resized_images, class_num])
    return np.array(image_data)


In [ ]:
# from tqdm.notebook import tqdm_notebook as tqdm_nb
# import time

train = get_data(train_image_dir)
val = get_data(val_image_dir)

In [ ]:
x_train = []
y_train = []
x_val = []
y_val = []

for feature, label in train:
    x_train.append(feature)
    y_train.append(label)

for feature, label in val:
    x_val.append(feature)
    y_val.append(label)



#### Normalizing the data

In [ ]:
# Normalize the data
x_train = np.array(x_train) / 255
x_val = np.array(x_val) / 255

x_train.reshape(-1, img_size, img_size, 1)
y_train = np.array(y_train)

x_val.reshape(-1, img_size, img_size, 1)
y_val = np.array(y_val)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        rotation_range = 30,
        zoom_range = 0.2, 
        width_shift_range=0.1,  
        height_shift_range=0.1, 
        horizontal_flip = True, 
        vertical_flip=False)


datagen.fit(x_train)

### Creating the model

In [ ]:
model = Sequential()

# Add first convo layer, maxpool and dropout
model.add(Conv2D(filters = 16, kernel_size = (5,5), 
                 activation = 'relu', input_shape = (img_size, img_size,3)))
model.add(MaxPooling2D(pool_size =(2,2)))
model.add(Dropout(0.25))

# Add a second convolution layer, maxpoling and dropout
model.add(Conv2D(filters = 32, kernel_size = (5,5), activation = 'relu'))
model.add(MaxPooling2D(pool_size =(2,2)))
model.add(Dropout(0.25))

# Add a third convolution layer, maxpoling and dropout
model.add(Conv2D(filters = 64, kernel_size = (5,5), activation = 'relu'))
model.add(MaxPooling2D(pool_size =(2,2)))
model.add(Dropout(0.25))


# Add a flattening layer
model.add(Flatten())

# add a layer with 5000 neurons and dropout
model.add(Dense(5000, activation = 'relu'))
model.add(Dropout(0.5))

# add a layer with 1000 neurons
model.add(Dense(1000, activation = 'relu'))
model.add(Dropout(0.5))

# add a layer with 500 neurons
model.add(Dense(500, activation = 'relu'))

model.add(Dropout(0.5))

# add a layer with 250 neurons
model.add(Dense(250, activation = 'relu'))

# add a layer with 10 neurons
model.add(Dense(10, activation = 'softmax'))

model.summary()

#### Model compile 

In [ ]:
from tensorflow.keras.optimizers import Adam

opt = Adam(learning_rate=0.000001)
model.compile(loss = 'sparse_categorical_crossentropy',
             optimizer = opt,
             metrics = ['accuracy'])

### Train the model

In [1]:
# Implement callback function to stop training  when accuracy reaches 96%
ACCURACY_THRESHOLD = 0.96

class myCallback(tf.keras.callbacks.Callback):
	def on_epoch_end(self, epoch, logs={}):
		if(logs.get('accuracy') > ACCURACY_THRESHOLD):
			print("\nReached %2.2f%% accuracy, so stopping training!!" %(ACCURACY_THRESHOLD*100))
			self.model.stop_training = True

# Instantiate a callback object
callbacks = myCallback()

hist = model.fit(x_train, y_train,
                 shuffle = True,
                 batch_size = 150,
                 epochs = 100,
                 validation_data = (x_val, y_val),
                 callbacks=[callbacks]
                )

NameError: name 'tf' is not defined

### Model evaluation

In [ ]:
performance = model.evaluate(x_val, y_val)[1]*100
performance

In [ ]:
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train','Val'], loc ='upper left')
plt.show()

In [ ]:
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train','Val'], loc ='upper right')
plt.show()

## Testing the model

In [ ]:
import os
!pip install opencv-python

import cv2
# !pip install scikit-image
from skimage.transform import resize

foldername = "./test_images/" ###### folder for test images

for img_count, filename in enumerate(os.listdir(foldername)):
    img = plt.imread(os.path.join(foldername,filename))
    resized_image = resize(img, (img_size, img_size, 3))
    plt.imshow(img)
    plt.show()
      
    predictions = model.predict(np.array([resized_image]))
    print(predictions)
    list_index = [0,1,2,3,4,5,6,7,8,9]
    x = predictions

  ################ display predictions as index from 0-9. ##############
    for i in range(len(labels)):
        for j in range(len(labels)):
            if x[0][list_index[i]] > x[0][list_index[j]]:
                #swap the numbers
                temp = list_index[i]
                list_index[i] = list_index[j]
                list_index[j] = temp
    display(list_index)
    print(' '*50)
    for i in range(len(labels)):
        print(labels[list_index[i]], ':', round((predictions[0][list_index[i]] * 100),2), '%')
    print("*" * 100)
